# ODE aktivity

In [1]:
from __future__ import division
import numpy as np
#%pylab inline
import matplotlib.pyplot as plt
# JSAnimation import available at https://github.com/jakevdp/JSAnimation
#from JSAnimation import IPython_display
from matplotlib import animation
import timeit

## Eulerův integrátor

## Keplerův problém

In [37]:
# Původní Viktorův algoritmus
# Modified (set of four first order differential equation) non-optimalized Euler 

# Set of initial declaration and parameters of the problem
a = 1.0
e = 0.01
GM = 4.0*np.pi**2

N = 400
t0 = 0.0
tend = 1.5
t = np.linspace(t0,tend,N)
h = (tend-t0)/N
x = np.zeros((np.size(t),4))


# Function calculate right sides of the ODEs
def f(x_local):
    radius = np.sqrt(x_local[0]**2+x_local[1]**2)
    tau = -GM/radius**3
    return np.array([h*x_local[2],h*x_local[3],h*tau*x_local[0],h*tau*x_local[1]])

# Euler integration method
def euler_vec(f, x , t, h):
    x[0,0] = 0.0
    x[0,1] = a*(1-e)
    x[0,2]= -np.sqrt(GM/a*(1+e)/(1-e))
    x[0,3] = 0.0
    
    x_buf = np.zeros(4)
    
    # Solution
    for i in range(N-2):
        x_buf[:] = x[i,:]
        x[i+1,:] = x_buf[:] + f(x_buf[:])
    return x

%timeit euler_vec(f,x,t,h)
solution = euler_vec(f,x,t,h)

plt.title('Trajectory of planet X')
plt.xlabel('X [1 AU]')
plt.ylabel('Y [1 AU]')
plt.plot(solution[:,0],solution[:,1],'k.')
plt.show()

100 loops, best of 3: 3.07 ms per loop


### Aktivity

* Vyzkoušejte si jaký vliv na přesnost řešení bude mít změna časového kroku $h$, tedy počtu bodů $N$. Vykreslete trajektorie planety pro různé hodnoty časového kroku do jednoho grafu. Algoritmus upravte tak, aby počet bodů byl stejný.
* Algoritmus prezentovaný výše není dobře vektorizován, pokuste se jej zefektivnit a porovnejte časy výpočtu

In [59]:
# Pozměněný Viktorův algoritmus
# Set of initial declaration and parameters of the problem
a = 1.0
e = 0.01
GM = 4.0*np.pi**2

N = 400
t0 = 0.0
tend = 1.5
#t = np.linspace(t0,tend,N)
h = (tend-t0)/N
x0 = np.array([[0.0, a*(1-e), -np.sqrt(GM/a*(1+e)/(1-e)), 0.0]])

# Function calculate right sides of the ODEs
def f(x_local):
    radius = np.sqrt(x_local[0]**2+x_local[1]**2)
    tau = -GM/radius**3
    return np.array([h*x_local[2], h*x_local[3], h*tau*x_local[0], h*tau*x_local[1]], float)

# Euler integration method
def euler_vec(f, x0, h): #t, h):
    
    # Solution
    x_buf = x0[:]
    for i in range(N-1):#N-2
        x_buf += f(x_buf[0])
        x0 = np.concatenate([x0, x_buf], axis=0)
    return x0.transpose()

#%timeit euler_vec(f,x0,h)
#solution = euler_vec(f,x0,h)

#plt.title('Trajectory of planet X')
#plt.xlabel('X [1 AU]')
#plt.ylabel('Y [1 AU]')
#plt.plot(solution[0],solution[1],'k.')
#plt.show()

# Euler integration method 2
def euler_vec2(f, x0, h): #t, h):
    x = np.zeros((N,4),dtype=float)
    
    # Solution
    x_buf = x0[:]#np.array([[0.0, a*(1-e), -np.sqrt(GM/a*(1+e)/(1-e)), 0.0]])
    for i in range(N):#N-2
        x[i,:] = x_buf[:]
        x_buf += f(x_buf[0])
    return x

%timeit euler_vec(f,x0,h)
x0 = np.array([[0.0, a*(1-e), -np.sqrt(GM/a*(1+e)/(1-e)), 0.0]])
%timeit euler_vec2(f,x0,h)
x0 = np.array([[0.0, a*(1-e), -np.sqrt(GM/a*(1+e)/(1-e)), 0.0]])
solution = euler_vec2(f,x0,h)

plt.title('Trajectory of planet X')
plt.xlabel('X [1 AU]')
plt.ylabel('Y [1 AU]')
plt.plot(solution[:,0],solution[:,1],'k.')
plt.show()

100 loops, best of 3: 4.09 ms per loop
100 loops, best of 3: 3.48 ms per loop


## Eulerova-Cromerova metoda

In [ ]:
#Viktorovo řešení
#  Non-optimalized Euler-Cronmer for Kepler problem

# Set of initial declaration and parameters of the problem
a = 1.0
e = 0.01
GM = 4.0*np.pi**2

N = 400
t0 = 0.0
tend = 1.5
t = np.linspace(t0,tend,N)
h = (tend-t0)/N
x = np.zeros((N,2))
v = np.zeros((N,2))


# Function g(x,t) 
def f(v_local):
    return np.array([h*v_local[0],h*v_local[1]])

# Function f(v,t) 
def g(x_local):
    radius = np.sqrt(x_local[0]**2+x_local[1]**2)
    tau = -GM/radius**3
    return np.array([h*tau*x_local[0],h*tau*x_local[1]])

# Euler integration method
def euler_vec(f, g, x, v, t, h):
    x[0,0] = 0.0
    x[0,1] = a*(1-e)
    v[0,0]= -np.sqrt(GM/a*(1+e)/(1-e))
    v[0,1] = 0.0
    
    x_buf = np.zeros(2)
    v_buf = np.zeros(2)
    
    # Solution
    for i in range(N-2):
        x_buf[:] = x[i,:]
        v_buf[:] = v[i,:]
        v[i+1,:] = v_buf[:] + g(x_buf[:])
        v_buf[:] = v[i+1,:]
        x[i+1,:] = x_buf[:] + f(v_buf[:])
    return x,v

solution_x,solution_v = euler_vec(f,g,x,v,t,h)

plt.title('Trajectory of planet X')
plt.xlabel('X [1 AU]')
plt.ylabel('Y [1 AU]')
plt.plot(solution_x[:,0],solution_x[:,1],'k.')

### Aktivity
* Úkol od Viktora: "Modifikujte Eulerovo-Cromerovo schéma tak, aby byla jako implicitní hodnota použita nikoliv rychlost, ale poloha!!!" Otestujte stabilitu této metody
* Modifikujte zadání *Keplerovy úlohy*, tak aby výsledný pohyb tělesa byl eliptický

In [ ]:
#Moje řešení
#vstupni konstanty
a = 1.0
e = 0.01
GM = 4.0*np.pi**2

#pocatecni podminky
kroku = 100
delka = 1.7
x = np.zeros(kroku)
y = np.zeros(kroku)
u = np.zeros(kroku)
v = np.zeros(kroku)
r = np.zeros(kroku)
T = np.zeros(kroku)
y[0] = a*(1-e)
u[0] = (-1)*sqrt((GM/a)*((1+e)/(1-e)))
r[0] = a*(1-e)
T[0] = -(GM/(r[0]**3))
               
#iterativni postup
for i in range(1,kroku):
    u[i] = u[i-1] + T[i-1]*x[i-1]*(delka/kroku)
    v[i] = v[i-1] + T[i-1]*y[i-1]*(delka/kroku)
    x[i] = x[i-1] + u[i]*(delka/kroku)
    y[i] = y[i-1] + v[i]*(delka/kroku)
    r[i] = sqrt(x[i-1]**2 + y[i-1]**2)
    T[i] = -(GM/(r[i-1]**3))
    
#Maticovy pristup
X = np.zeros(kroku, 6)

for j in range(6):
    
    X[j,i] = u

# Plotting of the result
plt.plot(x[:],y[:],"o-",color="red",lw=2)
plt.grid(True)
plt.xlabel("$x$ coordinate [AU]")
plt.ylabel("$y$ coordinate [AU]")

## Metody vyšších řádů
Úkol od Viktora - Runge-Kutta 2.řádu - Ralstonova metoda - použít na matematické kyvadlo.